# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
# Load All Necessary Imports
from statsmodels.regression.rolling import RollingOLS
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.api as sm
import warnings
import numpy as np
import quantstats as qs
import pandas_datareader.data as web
warnings.filterwarnings('ignore')

If Dataframe is in the data folder, gotta read from file in next lines. Since I'm lazy, I am using online data.

In [2]:
list_of_SP500_companies_wikipedia = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500 = pd.read_html(list_of_SP500_companies_wikipedia)
# 2 Dfs returned of which we are interested in the first
sp500 = sp500[0]

In [3]:
# Cleaning up the data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

# Check if changes came into effect
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 

Crucial moment. We have our tickers for the SP500 sorted, now we decide from what date the data can be counted as relevant. My argument is the market correction after the 2008 crash occured by logarithmic adjustment in August 2010 so we are approximating around then. There were no other global events except COVID that would decrease the quality of the data.

Problems:
Data might be too general
Long for the GARCH model to process
Hard to adjust for COVID year

In [4]:
end_date = dt.date.today()
start_date_15_years = pd.to_datetime(end_date) - pd.DateOffset(365*15)
start_date_10_years = pd.to_datetime(end_date) - pd.DateOffset(365*10)
start_date_5_years = pd.to_datetime(end_date) - pd.DateOffset(365*5)

In [5]:
# Downloads all the SNP500 Constituents

#15 years of data. Can be altered later to see whether predictability increases or decreases
df = yf.download(symbols_list, start=start_date_15_years, end=end_date)
df = df.stack()

[*********************100%***********************]  503 of 503 completed

1 Failed download:
['FIS']: ChunkedEncodingError(ProtocolError('Connection broken: IncompleteRead(7794 bytes read, 2446 more expected)', IncompleteRead(7794 bytes read, 2446 more expected)))


In [6]:
# Reorganize data
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2010-01-27 A        18.653919   20.865522   20.951359   20.550787   20.886980   
           AAPL      6.262736    7.424286    7.520714    7.126071    7.387500   
           ABT      18.608749   25.861219   25.976372   25.501369   25.957180   
           ACGL      7.588170    7.980000    7.995556    7.860000    7.895556   
           ACN      31.835499   41.930000   42.500000   41.529999   41.910000   
...                       ...         ...         ...         ...         ...   
2025-01-22 XYL     121.639999  121.639999  122.930000  121.040001  121.889999   
           YUM     126.220001  126.220001  126.779999  124.580002  125.360001   
           ZBH     109.720001  109.720001  111.150002  109.519997  110.250000   
           ZBRA    420.570007  420.570007  427.760010  419.589996  425.239990   
           ZTS     166.630005  166.630005  167.940002  164.729996  166.919998   

Price                    volume  
date       ticker                
2010-01-27 A       4.362179e+06  
           AAPL    1.722568e+09  
           ABT     2.410775e+07  
           ACGL    3.576600e+06  
           ACN     4.562300e+06  
...                         ...  
2025-01-22 XYL     1.191000e+06  
           YUM     2.498900e+06  
           ZBH     2.320700e+06  
           ZBRA    4.975000e+05  
           ZTS     1.610900e+06  

[1797654 rows x 6 columns]

Technical Indicators used for each stock: (Can be appended and changed later)
Garman-Klass Volatility
RSI
Bollinger Bands
ATR
MACD
Dollar Volume

In [7]:
# Insert Garman Class Volatility Column
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2 * np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open']) ** 2)
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2010-01-27 A        18.653919   20.865522   20.951359   20.550787   20.886980   
           AAPL      6.262736    7.424286    7.520714    7.126071    7.387500   
           ABT      18.608749   25.861219   25.976372   25.501369   25.957180   
           ACGL      7.588170    7.980000    7.995556    7.860000    7.895556   
           ACN      31.835499   41.930000   42.500000   41.529999   41.910000   
...                       ...         ...         ...         ...         ...   
2025-01-22 XYL     121.639999  121.639999  122.930000  121.040001  121.889999   
           YUM     126.220001  126.220001  126.779999  124.580002  125.360001   
           ZBH     109.720001  109.720001  111.150002  109.519997  110.250000   
           ZBRA    420.570007  420.570007  427.760010  419.589996  425.239990   
           ZTS     166.630005  166.630005  167.940002  164.729996  166.919998   

Price                    volume  garman_klass_vol  
date       ticker                                  
2010-01-27 A       4.362179e+06          2.437793  
           AAPL    1.722568e+09          0.837602  
           ABT     2.410775e+07          2.967229  
           ACGL    3.576600e+06          0.866607  
           ACN     4.562300e+06          4.053870  
...                         ...               ...  
2025-01-22 XYL     1.191000e+06          7.057288  
           YUM     2.498900e+06          7.147513  
           ZBH     2.320700e+06          6.728558  
           ZBRA    4.975000e+05         11.818092  
           ZTS     1.610900e+06          8.140637  

[1797654 rows x 7 columns]

In [8]:
# Insert RSI column with length = length
length = 20

def calculate_rsi(series, length):
    """ Returns the relative strength index of series based on given length (default is 20)"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=length).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=length).mean()
    rs = gain / loss
    rsi = 100 - 100 / (1 + rs)
    return rsi


df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: calculate_rsi(x, length))

# Plot RSU to make sure it works
df.xs('AAPL', level=1)['rsi'].plot()


<Axes: xlabel='date'>

In [13]:
# Adding Bollinger Bands
def bollinger_bands(series, length):
    """ Returns the Bollinger Bands of series based on given length (default is 20)"""
    dict = {}
    Simple_Moving_Avg = series.rolling(window=length).mean()
    Simple_Moving_Std = series.rolling(window=length).std()
    upper_band = Simple_Moving_Avg + 2 * Simple_Moving_Std
    lower_band = Simple_Moving_Avg - 2 * Simple_Moving_Std
    dict['u_bb'] = upper_band
    dict['l_bb'] = lower_band
    dict['sma'] = Simple_Moving_Avg
    return dict


df['upper_band'] = df.groupby(level=1)['adj close'].transform(lambda x: bollinger_bands(x, length)['l_bb'])

df['sma'] = df.groupby(level=1)['adj close'].transform(lambda x: bollinger_bands(x, length)['sma'])

df['lower_band'] = df.groupby(level=1)['adj close'].transform(lambda x: bollinger_bands(x, length)['u_bb'])

df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2010-01-27 A        18.653919   20.865522   20.951359   20.550787   20.886980   
           AAPL      6.262736    7.424286    7.520714    7.126071    7.387500   
           ABT      18.608749   25.861219   25.976372   25.501369   25.957180   
           ACGL      7.588170    7.980000    7.995556    7.860000    7.895556   
           ACN      31.835499   41.930000   42.500000   41.529999   41.910000   
...                       ...         ...         ...         ...         ...   
2025-01-22 XYL     121.639999  121.639999  122.930000  121.040001  121.889999   
           YUM     126.220001  126.220001  126.779999  124.580002  125.360001   
           ZBH     109.720001  109.720001  111.150002  109.519997  110.250000   
           ZBRA    420.570007  420.570007  427.760010  419.589996  425.239990   
           ZTS     166.630005  166.630005  167.940002  164.729996  166.919998   

Price                    volume  garman_klass_vol        rsi  upper_band  \
date       ticker                                                          
2010-01-27 A       4.362179e+06          2.437793        NaN         NaN   
           AAPL    1.722568e+09          0.837602        NaN         NaN   
           ABT     2.410775e+07          2.967229        NaN         NaN   
           ACGL    3.576600e+06          0.866607        NaN         NaN   
           ACN     4.562300e+06          4.053870        NaN         NaN   
...                         ...               ...        ...         ...   
2025-01-22 XYL     1.191000e+06          7.057288  64.610185  113.248349   
           YUM     2.498900e+06          7.147513  41.544117  120.920532   
           ZBH     2.320700e+06          6.728558  60.676351  101.470055   
           ZBRA    4.975000e+05         11.818092  67.336488  374.601077   
           ZTS     1.610900e+06          8.140637  55.519009  160.788981   

Price                     sma  lower_band         atr  standard_atr  
date       ticker                                                    
2010-01-27 A              NaN         NaN         NaN           NaN  
           AAPL           NaN         NaN         NaN           NaN  
           ABT            NaN         NaN         NaN           NaN  
           ACGL           NaN         NaN         NaN           NaN  
           ACN            NaN         NaN         NaN           NaN  
...                       ...         ...         ...           ...  
2025-01-22 XYL     117.304999  121.361649   87.153573     -0.130156  
           YUM     129.726500  138.532468   85.280716     -0.145415  
           ZBH     106.116887  110.763718   83.901431     -0.156652  
           ZBRA    395.406500  416.211924  102.740717     -0.003162  
           ZTS     164.557032  168.325083  111.349290      0.066976  

[1797654 rows x 13 columns]

In [12]:
# Implementing average true range
def compute_atr(stock_data, length=14):
    """ Calculates the True Range with default length of 14"""
    high_low = stock_data['high'] - stock_data['low']
    high_close = (stock_data['high'] - stock_data['close'].shift(1)).abs()
    low_close = (stock_data['low'] - stock_data['close'].shift(1)).abs()
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # Calculate ATR with a rolling window
    atr = true_range.rolling(window=length).mean()

    standardized_atr = atr.sub(atr.mean()).div(atr.std())

    return atr, standardized_atr

df['atr'], df['standard_atr'] = compute_atr(df)
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2010-01-27 A        18.653919   20.865522   20.951359   20.550787   20.886980   
           AAPL      6.262736    7.424286    7.520714    7.126071    7.387500   
           ABT      18.608749   25.861219   25.976372   25.501369   25.957180   
           ACGL      7.588170    7.980000    7.995556    7.860000    7.895556   
           ACN      31.835499   41.930000   42.500000   41.529999   41.910000   
...                       ...         ...         ...         ...         ...   
2025-01-22 XYL     121.639999  121.639999  122.930000  121.040001  121.889999   
           YUM     126.220001  126.220001  126.779999  124.580002  125.360001   
           ZBH     109.720001  109.720001  111.150002  109.519997  110.250000   
           ZBRA    420.570007  420.570007  427.760010  419.589996  425.239990   
           ZTS     166.630005  166.630005  167.940002  164.729996  166.919998   

Price                    volume  garman_klass_vol        rsi  upper_band  \
date       ticker                                                          
2010-01-27 A       4.362179e+06          2.437793        NaN         NaN   
           AAPL    1.722568e+09          0.837602        NaN         NaN   
           ABT     2.410775e+07          2.967229        NaN         NaN   
           ACGL    3.576600e+06          0.866607        NaN         NaN   
           ACN     4.562300e+06          4.053870        NaN         NaN   
...                         ...               ...        ...         ...   
2025-01-22 XYL     1.191000e+06          7.057288  64.610185  113.248349   
           YUM     2.498900e+06          7.147513  41.544117  120.920532   
           ZBH     2.320700e+06          6.728558  60.676351  101.470055   
           ZBRA    4.975000e+05         11.818092  67.336488  374.601077   
           ZTS     1.610900e+06          8.140637  55.519009  160.788981   

Price                     sma  lower_band         atr  standard_atr  
date       ticker                                                    
2010-01-27 A              NaN         NaN         NaN           NaN  
           AAPL           NaN         NaN         NaN           NaN  
           ABT            NaN         NaN         NaN           NaN  
           ACGL           NaN         NaN         NaN           NaN  
           ACN            NaN         NaN         NaN           NaN  
...                       ...         ...         ...           ...  
2025-01-22 XYL     117.304999  121.361649   87.153573     -0.130156  
           YUM     129.726500  138.532468   85.280716     -0.145415  
           ZBH     106.116887  110.763718   83.901431     -0.156652  
           ZBRA    395.406500  416.211924  102.740717     -0.003162  
           ZTS     164.557032  168.325083  111.349290      0.066976  

[1797654 rows x 13 columns]